# HTMX Integration

> HTMX patterns and helpers for daisyUI components in FastHTML

In [ ]:
#| default_exp core.htmx

In [ ]:
#| export
from typing import Dict, Any, Optional, List, Union, Literal, Callable
from dataclasses import dataclass, field
from enum import Enum
from fasthtml.common import *
from cjm_fasthtml_daisyui.core.base import DaisyComponent
from cjm_fasthtml_daisyui.core.colors import SemanticColor

## HTMX Integration Patterns

This module provides HTMX-aware patterns for daisyUI components, embracing the FastHTML philosophy of server-side rendering with progressive enhancement.

In [ ]:
#| export
class HTMXTrigger(str, Enum):
    """Common HTMX trigger events"""
    CLICK = "click"
    CHANGE = "change"
    SUBMIT = "submit"
    LOAD = "load"
    REVEALED = "revealed"
    INTERSECT = "intersect"
    EVERY = "every"
    KEYUP = "keyup"
    FOCUS = "focus"
    BLUR = "blur"
    
    def with_modifier(
        self,
        modifier: str  # TODO: Add description
    ) -> str:  # TODO: Add return description
        """Add modifier to trigger (e.g., 'click once')"""
        return f"{self.value} {modifier}"
    
    def delayed(
        self,
        delay: str  # TODO: Add description
    ) -> str:  # TODO: Add return description
        """Add delay to trigger (e.g., 'keyup delay:500ms')"""
        return f"{self.value} delay:{delay}"
    
    def changed(
        self
    ) -> str:  # TODO: Add return description
        """Add changed modifier (e.g., 'keyup changed')"""
        return f"{self.value} changed"

In [ ]:
#| export
class HTMXSwap(str, Enum):
    """HTMX swap strategies"""
    INNER_HTML = "innerHTML"
    OUTER_HTML = "outerHTML"
    BEFORE_BEGIN = "beforebegin"
    AFTER_BEGIN = "afterbegin"
    BEFORE_END = "beforeend"
    AFTER_END = "afterend"
    DELETE = "delete"
    NONE = "none"
    
    def with_modifier(
        self,
        modifier: str  # TODO: Add description
    ) -> str:  # TODO: Add return description
        """Add swap modifier (e.g., 'innerHTML swap:500ms')"""
        return f"{self.value} {modifier}"
    
    def with_transition(
        self,
        duration: str = "500ms"  # TODO: Add description
    ) -> str:  # TODO: Add return description
        """Add swap transition"""
        return f"{self.value} swap:{duration}"

In [ ]:
#| export
@dataclass
class HTMXAttrs:
    """Container for HTMX attributes"""
    hx_get: Optional[str] = None
    hx_post: Optional[str] = None
    hx_put: Optional[str] = None
    hx_patch: Optional[str] = None
    hx_delete: Optional[str] = None
    hx_trigger: Optional[str] = None
    hx_target: Optional[str] = None
    hx_swap: Optional[str] = None
    hx_indicator: Optional[str] = None
    hx_push_url: Optional[Union[bool, str]] = None
    hx_select: Optional[str] = None
    hx_select_oob: Optional[str] = None
    hx_vals: Optional[Union[str, Dict[str, Any]]] = None
    hx_confirm: Optional[str] = None
    hx_disable: Optional[bool] = None
    hx_disabled_elt: Optional[str] = None
    hx_include: Optional[str] = None
    hx_ext: Optional[str] = None
    
    def to_dict(
        self
    ) -> Dict[str, Any]:  # TODO: Add return description
        """Convert to dictionary of attributes"""
        attrs = {}
        for key, value in self.__dict__.items():
            if value is not None:
                # Convert Python snake_case to HTMX hyphenated attributes
                attr_name = key.replace('_', '-')
                
                # Handle boolean attributes
                if isinstance(value, bool):
                    attrs[attr_name] = str(value).lower()
                # Handle dictionary values (for hx-vals)
                elif isinstance(value, dict):
                    import json
                    attrs[attr_name] = json.dumps(value)
                else:
                    attrs[attr_name] = value
        
        return attrs

## HTMX-Aware Base Component

Extending DaisyComponent with HTMX support:

In [ ]:
#| export
class HTMXComponent(DaisyComponent):
    """
    Base class for HTMX-aware daisyUI components
    
    Extends DaisyComponent with HTMX attributes and helper methods
    for building interactive components following FastHTML patterns.
    """
    
    def __init__(self, *args, htmx: Optional[HTMXAttrs] = None, **kwargs):
        "TODO: Add function description"
        super().__init__(*args, **kwargs)
        self.htmx = htmx or HTMXAttrs()
    
    def with_htmx(
        self,
        get: Optional[str] = None,  # TODO: Add description
        post: Optional[str] = None,  # TODO: Add description
        put: Optional[str] = None,  # TODO: Add description
        patch: Optional[str] = None,  # TODO: Add description
        delete: Optional[str] = None,  # TODO: Add description
        trigger: Optional[Union[HTMXTrigger, str]] = None,
        target: Optional[str] = None,  # TODO: Add description
        swap: Optional[Union[HTMXSwap, str]] = None,
        **kwargs
    ) -> 'HTMXComponent':  # TODO: Add return description
        """
        Configure HTMX attributes fluently
        
        Args:
            get/post/put/patch/delete: URL endpoints
            trigger: Event that triggers the request
            target: CSS selector for target element
            swap: How to swap the response
            **kwargs: Additional HTMX attributes
            
        Returns:
            Self for method chaining
        """
        if get:
            self.htmx.hx_get = get
        if post:
            self.htmx.hx_post = post
        if put:
            self.htmx.hx_put = put
        if patch:
            self.htmx.hx_patch = patch
        if delete:
            self.htmx.hx_delete = delete
        
        if trigger:
            self.htmx.hx_trigger = trigger.value if isinstance(trigger, HTMXTrigger) else trigger
        if target:
            self.htmx.hx_target = target
        if swap:
            self.htmx.hx_swap = swap.value if isinstance(swap, HTMXSwap) else swap
        
        # Handle additional kwargs
        for key, value in kwargs.items():
            if hasattr(self.htmx, f"hx_{key}"):
                setattr(self.htmx, f"hx_{key}", value)
        
        return self
    
    def with_loading(
        self,
        indicator_id: str,  # TODO: Add description
        disable_during: Optional[str] = None  # TODO: Add description
    ) -> 'HTMXComponent':  # TODO: Add return description
        """
        Configure loading indicators
        
        Args:
            indicator_id: ID of the loading indicator element
            disable_during: CSS selector of elements to disable during request
            
        Returns:
            Self for method chaining
        """
        self.htmx.hx_indicator = f"#{indicator_id}"
        if disable_during:
            self.htmx.hx_disabled_elt = disable_during
        return self
    
    def with_confirmation(
        self,
        message: str  # TODO: Add description
    ) -> 'HTMXComponent':  # TODO: Add return description
        """
        Add confirmation dialog
        
        Args:
            message: Confirmation message to show
            
        Returns:
            Self for method chaining
        """
        self.htmx.hx_confirm = message
        return self
    
    def render_attrs(
        self
    ) -> Dict[str, Any]:  # TODO: Add return description
        """Build all HTML attributes including HTMX"""
        attrs = super().render_attrs()
        
        # Add HTMX attributes
        htmx_attrs = self.htmx.to_dict()
        attrs.update(htmx_attrs)
        
        return attrs

## Common HTMX Patterns

Pre-built patterns for common HTMX use cases:

In [ ]:
#| export
class HTMXPatterns:
    """Common HTMX patterns for daisyUI components"""
    
    @staticmethod
    def live_search(
        input_id: str,  # TODO: Add description
        results_id: str,  # TODO: Add description
        endpoint: str,  # TODO: Add description
        min_length: int = 2,  # TODO: Add description
        delay: str = "500ms"  # TODO: Add description
    ) -> Dict[str, Any]:  # TODO: Add return description
        """
        Create live search pattern
        
        Args:
            input_id: ID for the search input
            results_id: ID for the results container
            endpoint: Search endpoint URL
            min_length: Minimum characters before searching
            delay: Debounce delay
            
        Returns:
            Dictionary of attributes for the input element
        """
        return {
            "id": input_id,
            "type": "search",
            "hx-get": endpoint,
            "hx-trigger": HTMXTrigger.KEYUP.delayed(delay),
            "hx-target": f"#{results_id}",
            "hx-swap": HTMXSwap.INNER_HTML.value,
            "hx-indicator": f"#{results_id}-loading",
            "name": "q",
            "autocomplete": "off",
            "minlength": str(min_length)
        }
    
    @staticmethod
    def infinite_scroll(
        container_id: str,  # TODO: Add description
        next_page_endpoint: str,  # TODO: Add description
        threshold: str = "100px"  # TODO: Add description
    ) -> Dict[str, Any]:  # TODO: Add return description
        """
        Create infinite scroll pattern
        
        Args:
            container_id: ID of the container element
            next_page_endpoint: Endpoint for next page
            threshold: How far from bottom to trigger
            
        Returns:
            Dictionary of attributes for the sentinel element
        """
        return {
            "hx-get": next_page_endpoint,
            "hx-trigger": f"{HTMXTrigger.INTERSECT.value} once threshold:{threshold}",
            "hx-swap": HTMXSwap.AFTER_END.value,
            "hx-target": "this",
            "class": "htmx-indicator"
        }
    
    @staticmethod
    def click_to_edit(
        display_id: str,  # TODO: Add description
        edit_endpoint: str,  # TODO: Add description
        update_endpoint: str  # TODO: Add description
    ) -> Tuple[Dict[str, Any], Dict[str, Any]]:  # TODO: Add return description
        """
        Create click-to-edit pattern
        
        Args:
            display_id: ID of the display element
            edit_endpoint: Endpoint to get edit form
            update_endpoint: Endpoint to save changes
            
        Returns:
            Tuple of (display_attrs, form_attrs)
        """
        display_attrs = {
            "id": display_id,
            "hx-get": edit_endpoint,
            "hx-trigger": HTMXTrigger.CLICK.value,
            "hx-swap": HTMXSwap.OUTER_HTML.value,
            "class": "cursor-pointer hover:bg-base-200 p-2 rounded"
        }
        
        form_attrs = {
            "hx-post": update_endpoint,
            "hx-trigger": HTMXTrigger.SUBMIT.value,
            "hx-swap": HTMXSwap.OUTER_HTML.value
        }
        
        return display_attrs, form_attrs
    
    @staticmethod
    def delete_with_confirm(
        element_id: str,  # TODO: Add description
        delete_endpoint: str,  # TODO: Add description
        confirm_message: str = "Are you sure you want to delete this?"  # TODO: Add description
    ) -> Dict[str, Any]:  # TODO: Add return description
        """
        Create delete with confirmation pattern
        
        Args:
            element_id: ID of element to delete
            delete_endpoint: Delete endpoint URL
            confirm_message: Confirmation message
            
        Returns:
            Dictionary of attributes for delete button
        """
        return {
            "hx-delete": delete_endpoint,
            "hx-trigger": HTMXTrigger.CLICK.value,
            "hx-target": f"#{element_id}",
            "hx-swap": HTMXSwap.OUTER_HTML.with_transition(),
            "hx-confirm": confirm_message,
            "class": "btn btn-error btn-sm"
        }
    
    @staticmethod
    def poll_for_updates(
        element_id: str,  # TODO: Add description
        poll_endpoint: str,  # TODO: Add description
        interval: str = "2s"  # TODO: Add description
    ) -> Dict[str, Any]:  # TODO: Add return description
        """
        Create polling pattern for live updates
        
        Args:
            element_id: ID of element to update
            poll_endpoint: Endpoint to poll
            interval: Polling interval
            
        Returns:
            Dictionary of attributes
        """
        return {
            "id": element_id,
            "hx-get": poll_endpoint,
            "hx-trigger": f"{HTMXTrigger.EVERY.value} {interval}",
            "hx-swap": HTMXSwap.OUTER_HTML.value
        }

## Component-Specific HTMX Patterns

HTMX patterns for specific daisyUI components:

In [ ]:
#| export
class HTMXModal:
    """HTMX patterns for modal components"""
    
    @staticmethod
    def trigger_button(
        modal_id: str,  # TODO: Add description
        text: str,  # TODO: Add description
        endpoint: Optional[str] = None,  # TODO: Add description
        button_class: str = "btn btn-primary"  # TODO: Add description
    ) -> FT:  # TODO: Add return description
        """
        Create a button that opens a modal
        
        Args:
            modal_id: ID of the modal dialog element
            text: Button text
            endpoint: Optional endpoint to load modal content
            button_class: CSS classes for button
            
        Returns:
            Button element with proper attributes
        """
        attrs = {
            "class": button_class,
            "onclick": f"document.getElementById('{modal_id}').showModal()"
        }
        
        if endpoint:
            attrs.update({
                "hx-get": endpoint,
                "hx-target": f"#{modal_id} .modal-box",
                "hx-trigger": "click"
            })
        
        return Button(text, **attrs)
    
    @staticmethod
    def close_button(
        modal_id: str,  # TODO: Add description
        text: str = "Close",  # TODO: Add description
        button_class: str = "btn"  # TODO: Add description
    ) -> FT:  # TODO: Add return description
        """
        Create a close button for modal
        
        Args:
            modal_id: ID of the modal to close
            text: Button text
            button_class: CSS classes
            
        Returns:
            Close button element
        """
        return Form(
            Button(text, cls=button_class),
            method="dialog"
        )
    
    @staticmethod
    def async_modal(
        modal_id: str,  # TODO: Add description
        title: str,  # TODO: Add description
        content_endpoint: str  # TODO: Add description
    ) -> FT:  # TODO: Add return description
        """
        Create a modal that loads content asynchronously
        
        Args:
            modal_id: ID for the modal
            title: Modal title
            content_endpoint: Endpoint to load content from
            
        Returns:
            Modal dialog element
        """
        return Dialog(
            Div(
                H3(title, cls="font-bold text-lg"),
                Div(
                    Span("Loading...", cls="loading loading-spinner"),
                    id=f"{modal_id}-content",
                    cls="py-4",
                    hx_get=content_endpoint,
                    hx_trigger="load",
                    hx_swap="innerHTML"
                ),
                Form(
                    Button("Close", cls="btn"),
                    method="dialog",
                    cls="modal-action"
                ),
                cls="modal-box"
            ),
            Form(
                Button(cls="modal-backdrop"),
                method="dialog"
            ),
            id=modal_id,
            cls="modal"
        )

In [ ]:
#| export
class HTMXDropdown:
    """HTMX patterns for dropdown components"""
    
    @staticmethod
    def lazy_dropdown(
        dropdown_id: str,  # TODO: Add description
        button_text: str,  # TODO: Add description
        content_endpoint: str,  # TODO: Add description
        button_class: str = "btn"  # TODO: Add description
    ) -> FT:  # TODO: Add return description
        """
        Create dropdown that loads content on first open
        
        Args:
            dropdown_id: ID for the dropdown
            button_text: Text for dropdown button
            content_endpoint: Endpoint to load content
            button_class: CSS classes for button
            
        Returns:
            Dropdown element
        """
        return Details(
            Summary(button_text, cls=button_class),
            Ul(
                Li(
                    Span("Loading...", cls="loading loading-spinner loading-sm"),
                    cls="p-2"
                ),
                id=f"{dropdown_id}-content",
                cls="dropdown-content menu p-2 shadow bg-base-100 rounded-box w-52",
                hx_get=content_endpoint,
                hx_trigger="intersect once",
                hx_swap="innerHTML"
            ),
            id=dropdown_id,
            cls="dropdown"
        )
    
    @staticmethod
    def search_dropdown(
        dropdown_id: str,  # TODO: Add description
        search_endpoint: str,  # TODO: Add description
        placeholder: str = "Search..."  # TODO: Add description
    ) -> FT:  # TODO: Add return description
        """
        Create dropdown with search functionality
        
        Args:
            dropdown_id: ID for the dropdown
            search_endpoint: Endpoint for search
            placeholder: Search input placeholder
            
        Returns:
            Searchable dropdown element
        """
        return Details(
            Summary("Select", cls="btn"),
            Div(
                Input(
                    type="search",
                    placeholder=placeholder,
                    cls="input input-bordered input-sm w-full",
                    hx_get=search_endpoint,
                    hx_trigger=HTMXTrigger.KEYUP.delayed("300ms"),
                    hx_target=f"#{dropdown_id}-results",
                    name="q"
                ),
                Ul(
                    id=f"{dropdown_id}-results",
                    cls="menu mt-2"
                ),
                cls="dropdown-content p-2 shadow bg-base-100 rounded-box w-64"
            ),
            id=dropdown_id,
            cls="dropdown"
        )

In [ ]:
#| export
class HTMXForm:
    """HTMX patterns for form components"""
    
    @staticmethod
    def ajax_form(
        form_id: str,  # TODO: Add description
        endpoint: str,  # TODO: Add description
        method: str = "post",  # TODO: Add description
        success_target: Optional[str] = None,  # TODO: Add description
        loading_text: str = "Submitting..."  # TODO: Add description
    ) -> Dict[str, Any]:  # TODO: Add return description
        """
        Create AJAX form attributes
        
        Args:
            form_id: ID for the form
            endpoint: Form submission endpoint
            method: HTTP method
            success_target: Target for success response
            loading_text: Text to show while submitting
            
        Returns:
            Dictionary of form attributes
        """
        attrs = {
            "id": form_id,
            f"hx-{method}": endpoint,
            "hx-trigger": HTMXTrigger.SUBMIT.value,
            "hx-indicator": f"#{form_id}-indicator"
        }
        
        if success_target:
            attrs["hx-target"] = success_target
            attrs["hx-swap"] = HTMXSwap.INNER_HTML.value
        
        return attrs
    
    @staticmethod
    def inline_validation(
        input_id: str,  # TODO: Add description
        validation_endpoint: str,  # TODO: Add description
        error_target: str  # TODO: Add description
    ) -> Dict[str, Any]:  # TODO: Add return description
        """
        Create inline validation pattern
        
        Args:
            input_id: ID of input to validate
            validation_endpoint: Validation endpoint
            error_target: Where to show errors
            
        Returns:
            Dictionary of input attributes
        """
        return {
            "id": input_id,
            "hx-post": validation_endpoint,
            "hx-trigger": HTMXTrigger.BLUR.value,
            "hx-target": f"#{error_target}",
            "hx-swap": HTMXSwap.INNER_HTML.value,
            "hx-include": "this"
        }
    
    @staticmethod
    def submit_button_with_loading(
        text: str = "Submit",  # TODO: Add description
        loading_text: str = "Submitting...",  # TODO: Add description
        button_class: str = "btn btn-primary"  # TODO: Add description
    ) -> FT:  # TODO: Add return description
        """
        Create submit button with loading state
        
        Args:
            text: Button text
            loading_text: Text during loading
            button_class: CSS classes
            
        Returns:
            Button element with loading indicator
        """
        return Button(
            Span(text, cls="htmx-indicator-hide"),
            Span(
                Span(cls="loading loading-spinner loading-sm"),
                " ",
                loading_text,
                cls="htmx-indicator"
            ),
            type="submit",
            cls=button_class
        )

## Helper Functions

Utility functions for HTMX integration:

In [ ]:
#| export
def htmx_attrs(
    **kwargs
) -> Dict[str, Any]:  # Dictionary with proper HTMX attribute names
    "Convert keyword arguments to HTMX attributes Converts Python-style names to HTMX attribute names: - get -> hx-get - trigger -> hx-trigger - etc."
    attrs = {}
    for key, value in kwargs.items():
        # Convert to hx- prefix
        if not key.startswith('hx_'):
            key = f'hx_{key}'
        
        # Convert underscores to hyphens
        attr_name = key.replace('_', '-')
        
        # Handle special conversions
        if isinstance(value, HTMXTrigger):
            value = value.value
        elif isinstance(value, HTMXSwap):
            value = value.value
        elif isinstance(value, bool):
            value = str(value).lower()
        elif isinstance(value, dict):
            import json
            value = json.dumps(value)
        
        attrs[attr_name] = value
    
    return attrs

In [ ]:
#| export
def loading_indicator(
    indicator_id: str,  # ID for the indicator
    text: str = "Loading...",  # Loading text
    size: str = "md"  # Size of spinner (xs, sm, md, lg, xl)
) -> FT:  # Loading indicator element
    "Create a loading indicator element"
    return Div(
        Span(cls=f"loading loading-spinner loading-{size}"),
        " ",
        text,
        id=indicator_id,
        cls="htmx-indicator"
    )

In [ ]:
#| export
def oob_alert(
    message: str,  # Alert message
    alert_type: str = "info",  # Type (info, success, warning, error)
    target_id: str = "alerts",  # ID of container to append to
    auto_dismiss: Optional[int] = 5000  # Auto dismiss after milliseconds (None to disable)
) -> FT:  # Alert element with OOB swap
    "Create out-of-band alert message"
    alert_id = f"alert-{id(message)}"
    
    alert = Div(
        message,
        id=alert_id,
        cls=f"alert alert-{alert_type}",
        hx_swap_oob=f"beforeend:#{target_id}"
    )
    
    if auto_dismiss:
        alert.attrs[f"hx-trigger"] = f"load delay:{auto_dismiss}ms"
        alert.attrs["hx-swap"] = "outerHTML"
        alert.attrs["hx-target"] = f"#{alert_id}"
        alert.attrs["hx-get"] = "/empty"  # Endpoint that returns empty response
    
    return alert

## Usage Examples

Examples of using HTMX patterns with daisyUI components:

### Basic HTMX Component

In [ ]:
# Example: Create an HTMX-aware button
class HTMXButton(HTMXComponent):
    def component_class(self) -> str:
        return "btn"
    
    def supports_color(self) -> bool:
        return True

# Create button with HTMX behavior
button = HTMXButton(color=SemanticColor.PRIMARY)
button.with_htmx(
    get="/api/data",
    trigger=HTMXTrigger.CLICK,
    target="#results",
    swap=HTMXSwap.INNER_HTML
).with_loading("loading-spinner")

# Get the attributes
attrs = button.render_attrs()
print("Button attributes:")
for key, value in attrs.items():
    print(f"  {key}: {value}")

Button attributes:
  class: btn btn-primary
  hx-get: /api/data
  hx-trigger: click
  hx-target: #results
  hx-swap: innerHTML
  hx-indicator: #loading-spinner


### Using HTMX Patterns

In [ ]:
# Example: Live search pattern
search_attrs = HTMXPatterns.live_search(
    input_id="search-input",
    results_id="search-results",
    endpoint="/api/search",
    delay="300ms"
)

print("Live search attributes:")
for key, value in search_attrs.items():
    print(f"  {key}: {value}")

# Example: Delete with confirmation
delete_attrs = HTMXPatterns.delete_with_confirm(
    element_id="item-123",
    delete_endpoint="/api/items/123",
    confirm_message="Delete this item?"
)

print("\nDelete button attributes:")
for key, value in delete_attrs.items():
    print(f"  {key}: {value}")

Live search attributes:
  id: search-input
  type: search
  hx-get: /api/search
  hx-trigger: keyup delay:300ms
  hx-target: #search-results
  hx-swap: innerHTML
  hx-indicator: #search-results-loading
  name: q
  autocomplete: off
  minlength: 2

Delete button attributes:
  hx-delete: /api/items/123
  hx-trigger: click
  hx-target: #item-123
  hx-swap: outerHTML swap:500ms
  hx-confirm: Delete this item?
  class: btn btn-error btn-sm


### FastHTML Route Examples

In [ ]:
# Example FastHTML routes that work with HTMX patterns
from fasthtml.common import fast_app, serve

# app, rt = fast_app()

# Live search endpoint
# @rt('/api/search')
# def search(q: str = ""):
#     results = []  # Your search logic here
#     return Ul(
#         *[Li(A(result, href=f"/item/{result.id}")) for result in results],
#         cls="menu"
#     )

# Modal content endpoint
# @rt('/api/modal/user/{user_id}')
# def user_modal(user_id: int):
#     user = get_user(user_id)  # Your data logic
#     return Div(
#         H3(user.name, cls="font-bold text-lg"),
#         P(user.bio, cls="py-4"),
#         cls="space-y-4"
#     )

# Form submission with OOB alerts
# @rt('/api/form/submit')
# def submit_form(data: dict):
#     try:
#         # Process form
#         result = process_data(data)
#         
#         # Return success with OOB alert
#         return Div(
#             P("Form submitted successfully!"),
#             oob_alert("Data saved!", "success")
#         )
#     except Exception as e:
#         return oob_alert(str(e), "error")

print("Example FastHTML routes shown above (commented out)")

Example FastHTML routes shown above (commented out)


## Best Practices

1. **Server-Side First**: Embrace FastHTML's server-side rendering philosophy
2. **Progressive Enhancement**: Components work without JavaScript, HTMX enhances them
3. **Semantic HTML**: Use proper HTML elements (dialog, details, form)
4. **Loading States**: Always provide visual feedback during requests
5. **Error Handling**: Use OOB swaps for error messages
6. **Accessibility**: Ensure HTMX enhancements maintain accessibility

### Complete Example: Search with Modal

In [ ]:
# Complete example showing search with modal details
def create_search_interface():
    """Create a complete search interface with modal details"""
    return Div(
        # Search input
        Input(
            **HTMXPatterns.live_search(
                input_id="user-search",
                results_id="user-results",
                endpoint="/api/users/search"
            ),
            cls="input input-bordered w-full",
            placeholder="Search users..."
        ),
        
        # Results container
        Div(
            id="user-results",
            cls="mt-4"
        ),
        
        # Loading indicator
        loading_indicator("user-search-loading", "Searching..."),
        
        # Modal for user details
        HTMXModal.async_modal(
            modal_id="user-details-modal",
            title="User Details",
            content_endpoint="/api/users/{id}"
        ),
        
        # Alert container
        Div(id="alerts", cls="toast toast-end"),
        
        cls="max-w-lg mx-auto p-4"
    )

# Example search result item
def search_result_item(user_id: int, name: str, email: str):
    """Create a search result item with modal trigger"""
    return Li(
        A(
            Div(
                H4(name, cls="font-semibold"),
                P(email, cls="text-sm opacity-70"),
                cls="py-2"
            ),
            href="#",
            **htmx_attrs(
                get=f"/api/users/{user_id}",
                target="#user-details-modal .modal-box",
                trigger="click"
            ),
            onclick="user_details_modal.showModal()"
        )
    )

print("Complete search interface example created")

Complete search interface example created


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()